In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [3]:
class CaseModel(nn.Module):
    def __init__(self, embedding_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(CaseModel, self).__init__()    
        self.embedding_dim = embedding_dim
        self.hid1= embedding_dim*2
        self.hid2 = embedding_dim//2
        self.fc1 = nn.Linear(self.embedding_dim, self.hid1)
        self.fc2 = nn.Linear(self.hid1, self.hid2)
        self.hidden2tag = nn.Linear(self.hid2, tagset_size)
    
    def forward(self, x):
        y = F.relu(self.fc1(x))
        y = F.relu(self.fc2(y))
        y = self.hidden2tag( y )
        y = F.log_softmax(y, dim=1)
        return y

In [4]:
border=0.8

In [5]:
dict_path = "../models/utterance/"
dict_name = "group_border={0}_2.pickle".format(border)
dictM = DataManager(dict_path)
group_dict = dictM.load_data(dict_name)

success load : ../models/utterance/group_border=0.8_2.pickle


In [6]:
def search_word(word):
    word = clean_text(word)
    found_list = []
    for group_key in group_dict.keys():
        if word in group_dict[group_key]:
            found_list.append(group_key)
    return found_list

In [7]:
from gensim.models import KeyedVectors
w2v_path = "../../corpus/w2v/"
# fasttext
# https://qiita.com/Hironsan/items/513b9f93752ecee9e670
# w2v_name =  "dep-ja-300dim"
w2v_name =  "model.vec"
w2v_model = KeyedVectors.load_word2vec_format(w2v_path+w2v_name)

[8692] 2022-01-21 17:19:55,240 Info gensim.models.keyedvectors :loading projection weights from ../../corpus/w2v/model.vec
[8692] 2022-01-21 17:20:55,927 Info gensim.utils :KeyedVectors lifecycle event {'msg': 'loaded (351122, 300) matrix of type float32 from ../../corpus/w2v/model.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-01-21T17:20:55.926958', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-Ubuntu-18.04-bionic', 'event': 'load_word2vec_format'}


In [8]:
group2index = dict(zip( group_dict.keys(), range(len(group_dict.keys())) ))
index2group = dict(zip(range(len(group_dict.keys())), group_dict.keys() ))

In [9]:
model_path = "../models/utterance/"
model_name = "case_frame_{0}_2.pickle".format(border)
modelM = DataManager(model_path)
cmodel = modelM.load_data(model_name)

success load : ../models/utterance/case_frame_0.8_2.pickle


In [10]:
classify_border=0.75
def classify_word(word, mode="judge"):
    # word = clean_text(word)
    if word not in w2v_model:
        return ""
    with torch.no_grad():
        vector = torch.tensor([w2v_model[word]]).cuda()
        pred = np.exp(np.array(cmodel(vector).cpu()))
        p_argmax = pred.argmax()
        # print(pred[0][p_argmax])
        if mode=="register":
            if pred[0][p_argmax] < classify_border:
                return ""
    return index2group[p_argmax]

In [11]:
word =  "デジカメ"
classify_word(word)

'<もの>'

In [12]:
def register_triple(case_frame:dict, V, C, Noun):
    # 動詞の確認
    if V not in case_frame:
        case_frame[V] = dict()

    # 格の確認
    if C not in case_frame[V]:
        case_frame[V][C] = set()
    
    # 名詞の登録
    case_frame[V][C].add(Noun)

In [13]:
def compound_noun(token):
    base = token.lemma_
    for child in token.children:
        if child.dep_ == "compound":
           base = child.lemma_ + base
    return base

In [14]:
case_set = set("は　が　を　に　で")

def extract_RDF_triple(text, clean_=True):
    if clean_:
        text = clean_text(text)
    doc = nlp(text)
    triple_list = []
    for _, token in enumerate( doc ):
        if token.pos_ in ["VERB", "ADJ"]:
        # if token.pos_=="VERB":
        # if is_declinable(token):
            for c in token.children:
                if c.dep_ in ["nsubj", "obj", "obl"]:
                    noun = c.lemma_
                    for c2 in c.children:
                        if c2.dep_ == "case" and c2.orth_ in case_set:
                        # if c2.dep_ == "case":
                            case = c2.orth_
                            if case == "は":
                                case = "が"
                            triple_list.append( (token.lemma_, case, noun) )
    return triple_list

In [15]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [16]:
error = "Semantic error"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        # if ut.is_system() and ut.is_exist_error():
        if not ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [17]:
# with open("../../corpus/NTT/persona.json", "r", encoding="utf-8") as f:
#     convs = json.load(f)
# all_utt = []
# for did in tqdm( convs["convs"] ) :
#     dids = list( did.keys() )[0]
#     all_utt += did[dids]

In [18]:
# all_utt = sys_utt + all_utt
# len(all_utt)

In [19]:
def read_file(filename, batch_size=10000) -> list:
    f = open(filename, 'r', encoding='UTF-8')
    text_list = [  ]
    concated = ""
    for i, line in enumerate( f.readlines() ):
        concated += line.rstrip('\n')
        if (i+1) % batch_size == 0:
            text_list.append(concated)
            concated = ""
    text_list.append(concated)
    f.close()
    return text_list

In [20]:
filename = "../../corpus/wiki/wiki_normalized.txt"
filename = "../../corpus/wiki/wiki_40b_train_normal.txt"
# filename = "plain.txt"
all_list = read_file(filename, batch_size=100)
print("len of text_list :", len(all_list))

len of text_list : 18674


In [21]:
all_utt = all_list[::10]

In [22]:
case_frame = dict()

In [23]:
18674/6/60

51.87222222222223

In [24]:

for utt in tqdm( all_utt ):
    is_valid = True

    triples = extract_RDF_triple(utt, clean_=False)
    if len(triples)>0:
        for triple in triples:
            V = triple[0]
            C = triple[1]
            noun = triple[2]
            group_ = search_word(noun)
            if len(group_) > 0:
                for group in group_:
                    register_triple(case_frame, V, C, group)
            else:
                group = classify_word(noun, mode="register")
                # ないなら登録するな
                if group != "":
                    register_triple(case_frame, V, C, group)

            register_triple(case_frame, V, C, noun)


  1%|          | 19/1868 [02:42<4:23:59,  8.57s/it]


KeyboardInterrupt: 

In [25]:
case_frame_name = "case_frame_wiki40B_border={}.pickle".format(classify_border)
data_path = "../X_y_data/utterance/"

In [ ]:
# dataM = DataManager(data_path)
# dataM.save_data(case_frame_name, case_frame)

success save : ../X_y_data/utterance/case_frame_wiki40B_border=0.75.pickle


In [26]:
dataM = DataManager(data_path)
case_frame = dataM.load_data(case_frame_name)

success load : ../X_y_data/utterance/case_frame_wiki40B_border=0.75.pickle


In [35]:
def search_triple(V, C, N):
    if N in case_frame[V][C]:
        return True
    else:
        return False

# True : 用法は問題ない
# False : 問題あり
def judge_triple(triple):
    V = triple[0]
    C = triple[1]
    noun = triple[2]

    # Vが登録されていないならパスしておくかな(一旦)
    if V not in case_frame:
        # print("not registered V :", V)
        return True
    
    # C が登録されていないなら，アウト
    if C not in case_frame[V]:
        print("not registered V, C :", V, C)
        return False
    
    # そのままの名詞で検索 -> 発見なら正例
    if search_triple(V, C, noun):
        return True

    # 名詞がどこかのグループに属する
    group_ = search_word(noun)
    if len(group_) > 0:
        # 1つでも引っかかればOK
        for group in group_:
            if search_triple(V, C, group):
                return True
        return False
    # グループを推定
    else:
        group = classify_word(noun) 
        return search_triple(V, C, group)

In [36]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

error = "Semantic error"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [29]:
y_pred = []
for utt in tqdm(sys_utt):
    is_valid = True

    triples = extract_RDF_triple(utt)
    for triple in triples:
        if not judge_triple(triple):
            print(triple)
            is_valid = False
            break
    if is_valid:
        y_pred.append(0)
    else:
        y_pred.append(1)

  1%|          | 15/1386 [00:00<00:19, 70.08it/s]

('生まれる', 'に', 'お兄さん')
('見る', 'を', '好き')


  3%|▎         | 43/1386 [00:00<00:16, 81.53it/s]

('ほしい', 'が', '帽子')
('大好き', 'が', '帽子')
('得意', 'が', 'クロール')
('泳ぐ', 'で', 'スイミング')
('楽しい', 'が', 'スイミング')


  4%|▍         | 60/1386 [00:00<00:17, 74.03it/s]

...

('不便', 'が', '手汗')
('高い', 'が', '冬服')
('育つ', 'で', 'さん')


 99%|█████████▉| 1378/1386 [00:23<00:00, 77.21it/s]

('整理', 'が', '上')
('確保', 'が', '踏み場')
('整理', 'が', '上')
('確保', 'が', '踏み場')
('みかける', 'で', '図書館')


100%|██████████| 1386/1386 [00:23<00:00, 58.23it/s]

('経つ', 'に', 'たち')


In [30]:
score(y, y_pred)

confusion matrix = 
 [[1089  289]
 [   6    2]]
accuracy =  0.7871572871572872
precision =  0.006872852233676976
recall =  0.25
f1 score =  0.013377926421404684


In [31]:
word = "一人"
print("group "+classify_word(word))
print(word in w2v_model)

group <ヒト（数）>
True


In [37]:
case_frame["整理"]

{'に': {'0',
  '00',
  '<お金（接尾辞）>',
  '<イベント>',
  '<ヒト>',
  '<ヒト（役割）>',
  '<ヒト（組織）>',
  '<ヒト（要素）>',
  '<位置>',
  '<作業>',
  '<傾向>',
  '<副詞>',
  '<単位>',
  '<変化>',
  '<学問>',
  '<家具>',
  '<建物>',
  '<指示代名詞>',
  '<敬称>',
  '<数字>',
  '<文字>',
  '<文書>',
  '<時期>',
  '<時間（単位）>',
  '<植物>',
  '<機械>',
  '<状況>',
  '<管理行動>',
  '<考え>',
  '<複文>',
  '<規則>',
  '<量>',
  'この頃',
  'これ',
  'ごと',
  'つ',
  'とも',
  'アレン',
  'カード',
  'グレード',
  'コンピュータ',
  'コード',
  'トランザクション',
  'ドル',
  'ハドソン',
  '中心',
  '代わり',
  '以外',
  '余',
  '優子',
  '刑法学',
  '別',
  '博物館',
  '史',
  '同時',
  '坊',
  '基礎',
  '寺',
  '年',
  '形',
  '後',
  '悪影響',
  '成立',
  '手',
  '支城',
  '改正',
  '方',
  '本',
  '東久宝寺',
  '根拠',
  '植民地',
  '概念',
  '次第',
  '状',
  '番号',
  '種',
  '簡体字',
  '組織',
  '英松',
  '著書',
  '解析',
  '設計',
  '連番',
  '週',
  '部'},
 'が': {'<もの>',
  '<イベント>',
  '<スポーツ>',
  '<ヒト>',
  '<ヒト（役割）>',
  '<ヒト（組織）>',
  '<乗り物>',
  '<作業>',
  '<傾向>',
  '<副詞>',
  '<単位>',
  '<場所>',
  '<場所（接尾辞）>',
  '<変化>',
  '<存在・関係>',
  '<建物>',
  '<建物（接尾辞）>',
  '<心理>',
  '<敬称>

- 学習データのみでやってみた

        confusion matrix = 
        [[716 662]
        [  1   7]]
        accuracy =  0.5216450216450217
        precision =  0.01046337817638266
        recall =  0.875
        f1 score =  0.020679468242245203

    - 再現率は良い！

In [33]:
def judge_triple(triple):
    V = triple[0]
    C = triple[1]
    noun = triple[2]

    # Vが登録されていないならパスしておくかな(一旦)
    if V not in case_frame:
        # print("not registered V :", V)
        return True
    
    # C が登録されていないなら，アウト
    if C not in case_frame[V]:
        # print("not registered V, C :", V, C)
        return False
    
    # そのままの名詞で検索 -> 発見なら正例
    if search_triple(V, C, noun):
        # print("\tfound", V, C, noun)
        return True

    # 名詞がどこかのグループに属する
    group_ = search_word(noun)
    if len(group_) > 0:
        # 1つでも引っかかればOK
        for group in group_:
            if search_triple(V, C, group):
                return True
        return False
    # グループを推定
    else:
        group = classify_word(noun) 
        return search_triple(V, C, group)

In [38]:
for yt, yp, utt in zip(y, y_pred, sys_utt):
    if yt==1:
        print(utt)
        triples = extract_RDF_triple(utt)
        for triple in triples:
            print(triple)
            print( judge_triple(triple) )
        print("--------")

元気ですかは元気です
--------
好きだを見ますよねー
('見る', 'を', '好き')
False
--------
病院は治療を受けましょう
('受ける', 'が', '病院')
True
('受ける', 'を', '治療')
True
--------
好きだは好きですか。お寿司はエンガワが好きですね
('好き', 'が', '縁側')
False
--------
時期から資格を取りますねぇ
('取る', 'を', '資格')
True
--------
手を貯金に出しますねぇ
('出す', 'を', '手')
True
('出す', 'に', '貯金')
True
--------
ところで、テレビでテレビあるって言ってましたが、テレビは民主党支持が多いですね
('ある', 'で', 'テレビ')
True
('言う', 'で', '所')
True
('多い', 'が', '支持')
True
--------
旬ですねぇ。自分もオリンピック書いたし。
--------
